# How to pick the best strike and expiration for trading options

## Parse historic options data

## Load data into Optopsy

## Analyze strategy performance

In [10]:
# import and setup
import os
from pathlib import Path
import glob
import pandas as pd
import optopsy as op

In [26]:
files = glob.glob(str(Path(Path(os.getcwd()).parents[0], "data", os.path.join("rut-eod", "*.csv"))))
dfs = []
for fl in files:
    df = pd.read_csv(fl)
    dfs.append(df)
pd.concat(dfs).to_csv(Path(Path(os.getcwd()).parents[0], "data","rut_historic.csv"), index=False)

In [28]:
# Loading data into Optopsy
rut_chains = op.csv_data(
    Path(Path(os.getcwd()).parents[0], "data","rut_historic.csv"),
    underlying_symbol=1,
    underlying_price=4,
    option_type=8,
    expiration=6,
    quote_date=0,
    strike=7,
    bid=14,
    ask=15
)

/opt/homebrew/Caskroom/miniforge/base/envs/py311/lib/python3.11/site-packages/optopsy/datafeeds.py:42: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data["expiration"] = pd.to_datetime(data.expiration, infer_datetime_format=True)
/opt/homebrew/Caskroom/miniforge/base/envs/py311/lib/python3.11/site-packages/optopsy/datafeeds.py:43: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data["quote_date"] = pd.to_datetime(data.quote_date, infer_datetime_format=True)


In [ ]:
rut_chains

In [ ]:
# Analyze strategies with Optopsy
op.short_calls(rut_chains).round(2)